In [1]:

from skimage import transform
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns


In [2]:
data_folder = 'data/cubicasa5k/'
data_file = 'val.txt'
training_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
rooms = ["Outdoor", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Storage", "Garage", "Undefined"]
room_ids = [1,3,4,5,6,7,9,10,11]
room_labels = {rooms[i]:i+1 for i in range(len(rooms))}

In [28]:
import copy
import cv2
from tqdm import tqdm_notebook as tqdm
instance_info = {}

for i in tqdm(range(len(training_set))):
    item = training_set.get_txt(i)
    masks = item['label'][0].data.numpy()
    
    H,W = masks.shape
    boxes = []
    labels = []
    num_obj = 0
    
    mask_tensor = []
    areas = []
    
    for r in room_ids:
        x = copy.copy(masks)
        x[masks != r] = 0 
        x = x.astype(np.uint8)
        #plt.imshow(x)
        #plt.show()
        contours, _ = cv2.findContours(x,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        
        for tcnt in contours: 
            
            img = np.zeros((H,W,3), np.uint8)
            img = cv2.drawContours(img, [tcnt], -1, (255,255,255), -1)
            mask_tensor.append((img[:,:,0]/255).astype(np.int8))
            areas.append(cv2.contourArea(tcnt,False) )
            x,y,w,h = cv2.boundingRect(tcnt)
            boxes.append([x,y,x+w,y+h])
            labels.append(room_labels[room_classes[r]])
            num_obj+=1
    
    boxes = torch.FloatTensor(boxes)
    labels = torch.FloatTensor(labels).int()
    image_id = torch.ones(1, dtype=torch.uint8)*i
    areas = torch.FloatTensor(areas)
    
    try:
        mask_tensor = torch.from_numpy(np.stack(mask_tensor, 0))
    except:
        mask_tensor = torch.FloatTensor([])
    
        
    
    instance_info[item['folder']] ={'boxes': boxes, 'labels':labels, 'image_id':image_id,
                                'area':areas, 'iscrowd':torch.zeros(num_obj,dtype=torch.uint8) ,\
                                'masks': mask_tensor}
    
    
    
    
#save the file
torch.save(instance_info, 'data/cubicasa5k/instance_info_val.pt')



In [4]:
import copy
import cv2
from tqdm import tqdm_notebook as tqdm

data_file = 'train.txt'
training_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
for i in tqdm(range(len(training_set))):
    item = training_set.get_txt(i)
    masks = item['label'][0].data.numpy()
    
    
    num_obj = 0
    
   
    
    for r in room_ids:
        x = copy.copy(masks)
        x[masks != r] = 0 
        x = x.astype(np.uint8)
        #plt.imshow(x)
        #plt.show()
        contours, _ = cv2.findContours(x,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        num_obj+=len(contours)
        
    print(num_obj)
    
    
    



10
26
5
19
17
29
11
10
16
4
16
7
19
21
10
18
8
12
14
24
21
12
19
12
7
12
14
6
7
16
9
12
10
10
17
11
19
17
7
25
10
24
9
23
19
18
6
18
10
14
18
6
7
13
18
5
29
17
12
7
15
11
19
18
7
11
21
5
27
5
16
7
31
20
35
15
10
31
14
16
7
13
24
21
6
11
25
15
13
22
22
6
11
18
11
15
40
9
18
17
19
9
7
7
4
6
12
5
6
9
24
15
25
6
9
18
4
10
7
18
2
18
9
13
7
38
19
8
23
6
25
10
5
5
4
12
10
31
11
10
14
14
19
6
19
7
23
6
14
14
13
5
19
15
7
9
20
13
18
16
10
11
13
35
5
15
5
19
7
17
10
4
11
10
15
16
12
15
15
22
19
21
22
22
21
7
29
21
8
8
18
12
15
2
21
7
9
9
7
30
6
8
6
13
12
13
2
7
12
21
11
19
8
10
8
27
7
17
26
11
13
7
9
8
20
9
21
12
6
10
7
11
12
7
5
12
9
8
8
23
8
7
13
10
18
12
13
12
9
5
14
6
13
6
11
13
21
6
25
20
11
9
17
13
8
7
7
6
7
19
9
11
4
7
22
14
8
13
5
6
9
21
21
8
9
10
16
6
20
27
6
12
6
19
13
13
7
12
15
18
11
6
6
18
22
14
8
9
7
8
6
18
14
15
13
6
16
5
15
3
11
15
5
25
10
25
15
9
14
9
16
9
15
8
7
3
9
6
24
16
5
16
17
15
6
10
16
12
11
15
18
16
6
4
7
9
28
17
16
12
17
9
16
10
9
20
6
9
13
18
8
24
17
17
20
7
11
15
9
1

KeyboardInterrupt: 